# Long Term Open Source Metric Analysis

# Hadoop Analysis

In [76]:
hadoop_versions = [
         "2.9.2", "2.9.1", "2.9.0","2.8.5", "2.8.4", "2.8.3","2.8.2", "2.8.1","2.8.0",
        "2.7.7","2.7.6","2.7.5","2.7.3","2.7.2","2.7.1","2.7.0","2.6.5", "2.6.4","2.6.3"
        ,"2.6.2","2.6.1","2.6.0","2.5.2","2.5.1","2.5.0","2.4.1","2.4.0","2.3.0"
        ,"3.3.0", "3.3.1", "3.3.2", "3.3.3", "3.3.4", "3.3.5","3.3.6"
       ,"3.2.0", "3.3.1", "3.2.2", "3.2.3", "3.2.4"
       ,"3.1.0","3.1.1","3.1.2","3.1.3","3.1.4"
       ,"3.0.0","3.0.1","3.0.2","3.0.3"
    ]


neededColumnsClass =['cbo','cboModified','fanin','fanout','wmc','dit','noc','lcom',
                     'lcom*','tcc','lcc', 'loc'
               ]


In [77]:
len(hadoop_versions)

49

In [78]:
import pandas as pd

classCkMetricsAllVersionsDfClass = pd.read_csv('DataSource/hadoop_AllCkMetricsAllVersionsClass.csv')

In [79]:
hadoopBugsAllDf = pd.read_csv('DataSource/hadoop_bugs_all.csv')
hadoopNewFeaturesAllDf = pd.read_csv('DataSource/hadoop_newFeature_all.csv')
hadoopImprovementsAllDf = pd.read_csv('DataSource/hadoop_improvements_all.csv')

In [80]:
cleanedCkMetricsAllVersionsDf = (classCkMetricsAllVersionsDfClass
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("Test")]
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("test")]
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("TEST")]
    .loc[~classCkMetricsAllVersionsDfClass['file'].str.contains("src/test/")]
) 

In [81]:
print("Filtrelenmiş Sınıf Sayısı:", cleanedCkMetricsAllVersionsDf.shape[0])
print("Versionlanmış hata sayısı:", hadoopBugsAllDf.shape[0])
print("Versionlanmış yeni özellik sayısı:", hadoopImprovementsAllDf.shape[0])
print("Versionlanmış geliştirme sayısı:", hadoopNewFeaturesAllDf.shape[0])
cleanedCkMetricsAllVersionsDf.head()

Filtrelenmiş Sınıf Sayısı: 445474
Versionlanmış hata sayısı: 21640
Versionlanmış yeni özellik sayısı: 3139
Versionlanmış geliştirme sayısı: 258


,Unnamed: 0,Version,file,class,type,cbo,cboModified,fanin,fanout,wmc,...,assignmentsQty,mathOperationsQty,variablesQty,maxNestedBlocksQty,anonymousClassesQty,innerClassesQty,lambdasQty,uniqueWordsQty,modifiers,logStatementsQty
0,0,2.9.2,/Users/ayferuckan/Desktop/TezÇalışmam/HadoopMe...,org.apache.hadoop.mapreduce.v2.app.rm.RMHeartb...,interface,0,3,3,0,2,...,0,0,0,0,0,0,0,12,1,0
3,3,2.9.2,/Users/ayferuckan/Desktop/TezÇalışmam/HadoopMe...,org.apache.hadoop.yarn.server.federation.store...,class,3,9,6,3,1,...,1,0,1,0,0,0,0,29,1025,0
4,4,2.9.2,/Users/ayferuckan/Desktop/TezÇalışmam/HadoopMe...,org.apache.hadoop.fs.FileSystem,class,63,190,127,63,278,...,116,47,94,6,6,5,0,1040,1025,18
5,5,2.9.2,/Users/ayferuckan/Desktop/TezÇalışmam/HadoopMe...,org.apache.hadoop.hdfs.DFSUtil$ConfiguredNNAdd...,innerclass,0,0,0,0,5,...,3,1,3,0,0,0,0,22,9,0
6,6,2.9.2,/Users/ayferuckan/Desktop/TezÇalışmam/HadoopMe...,org.apache.hadoop.mapreduce.lib.output.Sequenc...,innerclass,2,2,0,2,6,...,3,0,1,0,0,0,0,29,9,0


In [82]:


hadoopBugsGroupedDf = hadoopBugsAllDf.groupby(['AVersion']).count().reset_index()
hadoopImprovementsGroupedDf= hadoopImprovementsAllDf.groupby(['Fix Version/s']).count().reset_index()
hadoopNewFeaturesGroupedDf= hadoopNewFeaturesAllDf.groupby(['Fix Version/s']).count().reset_index()

hadoopBugsGroupedDf = hadoopBugsGroupedDf.rename(columns = {'AVersion': 'Version','Issue key': 'Count'})[['Version','Count']]
hadoopImprovementsGroupedDf = hadoopImprovementsGroupedDf.rename(columns = {'Fix Version/s': 'Version','Issue key': 'Count'})[['Version','Count']]
hadoopNewFeaturesGroupedDf = hadoopNewFeaturesGroupedDf.rename(columns = {'Fix Version/s': 'Version','Issue key': 'Count'})[['Version','Count']]

metricsAllVersionsDf = pd.merge(cleanedCkMetricsAllVersionsDf, hadoopBugsGroupedDf,on="Version", how='inner')
ckMetricsAllVersionsDf = metricsAllVersionsDf.dropna()
ckMetricsAllVersionsDf.columns = ckMetricsAllVersionsDf.columns.str.upper()

In [108]:
vckMetricsAllVersionsDf = ckMetricsAllVersionsDf
vckMetricsAllVersionsDf.loc[:, 'NUMERICVERSION']  = ckMetricsAllVersionsDf['VERSION'].apply(lambda x: int(''.join(x.split('.'))))

selected_columns = ['CBO', 'CBOMODIFIED', 'FANIN', 'FANOUT', 'WMC', 'DIT', 'NOC', 'LCOM',
       'LCOM*', 'TCC', 'LCC', 'LOC', 'NUMERICVERSION','COUNT' ]
df = vckMetricsAllVersionsDf[selected_columns]
df.head()

,CBO,CBOMODIFIED,FANIN,FANOUT,WMC,DIT,NOC,LCOM,LCOM*,TCC,LCC,LOC,NUMERICVERSION,COUNT
0,0,3,3,0,2,1,0,1,0.000000,0.000000,0.000000,4,292,34
2,63,190,127,63,278,3,15,14945,0.985463,0.002395,0.004649,1510,292,34
3,0,0,0,0,5,1,0,0,0.400000,0.500000,1.000000,22,292,34
4,2,2,0,2,6,1,0,0,0.166667,0.666667,0.666667,21,292,34
5,3,3,0,3,2,2,0,1,0.000000,0.000000,0.000000,8,292,34


In [109]:
from sklearn.model_selection import train_test_split
X = df.drop('COUNT', axis=1)  
y = df['COUNT']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,random_state=42)

In [110]:
X_train.shape

(254360, 13)

In [111]:
import tensorflow as tf
tf.random.set_seed(42)

normalization_layer = tf.keras.layers.Normalization()         #(1)
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")  #(2)
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()                  #(3)
output_layer = tf.keras.layers.Dense(1)                       #(4)



input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

In [112]:
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 normalization_4 (Normalization  (None, 13)          27          ['input_5[0][0]']                
 )                                                                                                
                                                                                                  
 dense_12 (Dense)               (None, 30)           420         ['normalization_4[0][0]']        
                                                                                                  
 dense_13 (Dense)               (None, 30)           930         ['dense_12[0][0]']         

In [113]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)

model.compile(
    loss="mse", 
    optimizer=optimizer, 
    metrics=["RootMeanSquaredError"])

In [114]:


normalization_layer.adapt(X_train)

In [115]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
6359/6359 [==============================] - 3s 419us/step - loss: 41288.7578 - root_mean_squared_error: 203.1964 - val_loss: 39499.0000 - val_root_mean_squared_error: 198.7436
Epoch 2/20
6359/6359 [==============================] - 3s 407us/step - loss: 39200.2812 - root_mean_squared_error: 197.9906 - val_loss: 36766.5742 - val_root_mean_squared_error: 191.7461
Epoch 3/20
6359/6359 [==============================] - 3s 407us/step - loss: 36185.8984 - root_mean_squared_error: 190.2259 - val_loss: 34395.8906 - val_root_mean_squared_error: 185.4613
Epoch 4/20
6359/6359 [==============================] - 3s 407us/step - loss: 34755.9609 - root_mean_squared_error: 186.4295 - val_loss: 33034.7070 - val_root_mean_squared_error: 181.7545
Epoch 5/20
6359/6359 [==============================] - 3s 406us/step - loss: 32352.6562 - root_mean_squared_error: 179.8684 - val_loss: 29367.8750 - val_root_mean_squared_error: 171.3706
Epoch 6/20
6359/6359 [==============================] - 3s 4

In [74]:
mse_test = model.evaluate(X_test, y_test)


2650/2650 [==============================] - 1s 273us/step - loss: 40578.9961 - root_mean_squared_error: 201.4423
